In [14]:
import numpy as np
import scipy
from scipy import ndimage
from scipy import misc
import pickle
import time
import matplotlib.pyplot as plt
import cv2
import os

import tensorflow as tf
from tensorflow import keras

from ABRS_modules import discrete_radon_transform
from ABRS_modules import create_ST_image
from ABRS_modules import smooth_2d
from ABRS_modules import center_of_gravity
#from ABRS_modules import subplot_images
from ABRS_modules import getting_frame_record
from ABRS_modules import read_frames
from ABRS_modules import read_frames2
from ABRS_modules import subtract_average
from ABRS_modules import create_3C_image



def video_clips_to_3C_image_fun (dirPathInput,dirPathOutput,fbList,clipStart,clipEnd,clipsNumber,bufferSize,windowST,modelName,OSplatform='winds'):

    #OSplatform = 'winds'
    
    if modelName != 'none':
    
        #model = keras.models.load_model('modelConv2ABRS_3C_LiManualLabel_fullTraining_norm256')
        #model = keras.models.load_model('modelConv2ABRS_3C_train_with_labelCS1fb1_SS_plus_LiManualLabel_3')
        model = keras.models.load_model(modelName)

    clips = np.arange(clipFirst,clipsNumber,1);

    fileList = os.listdir(dirPathInput);

    clIndex = 0;

    for cl in clips:


        fileName = fileList[cl];

        ext = fileName[-3:];

        if (ext == 'avi' or ext == 'mov') == True:

            fileDirPathInputName = dirPathInput + '\\' + fileName;

            r = np.arange(0,clipEnd,bufferSize)

            for bf in r:

                    startFrame = bf;
                    endFrame = startFrame + bufferSize;
                        
                    frRec = read_frames(startFrame, endFrame, fileDirPathInputName, [400,400])

                    if clIndex == clips[0] and startFrame == clipStart:
                        frRecRemain = np.zeros((windowST,160000));
                        frRec = np.concatenate((frRecRemain,frRec), axis=0);

                    if clIndex > clips[0] or startFrame > clipStart:
                        frRec = np.concatenate((frRecRemain,frRec), axis=0);  

                    for fb in fbList:

                        recIm3C = np.zeros((50,80,80,3))

                        for w in range(0,frRec.shape[0]-windowST):

                            startWin = w;
                            endWin = startWin + windowST;
                       
                            posDic, maxMovement, cfrVectRec, frameVectFloatRec = getting_frame_record(frRec, startWin, endWin,fb)


                            im3C = create_3C_image (cfrVectRec)
                            #print(im3C[30,30,2])
                            
                            if modelName != 'none':
                                predictBehavior = 1  # predict behavior from ST-image and store the label
                            else:
                                predictBehavior = 0
                            
                            if predictBehavior == 1:
                                                                
                                X_rs = np.zeros((1,80,80,3))
        
                                X_rs[0,:,:,:]=im3C

                                X = X_rs/256  # normalize

                                predictionsProb = model.predict(X)
                                predictionLabel = np.zeros((1,np.shape(predictionsProb)[0]))
                                predictionLabel[0,:] = np.argmax(predictionsProb,axis=1)
                    
                    

                            cv2.imshow('im3C',im3C)

                            recIm3C[w,:,:,:]=im3C
                            #print(np.shape(recIm3C))

                            xPos = posDic["xPos"];
                            yPos = posDic["yPos"];
                                                       

                            if w == 0:
                            
                                #sMRec = sMNorm;
                                xPosRec = xPos;
                                yPosRec = yPos;
                                maxMovementRec = maxMovement
                                    
                                if modelName != 'none': 
                                       behPredictionRec = predictionLabel 
                                else: behPredictionRec = 0        
                                
                            if w > 0:
                                
                                #sMRec = np.vstack((sMRec,sMNorm));
                                xPosRec = np.vstack((xPosRec,xPos));
                                yPosRec = np.vstack((yPosRec,yPos));
                                maxMovementRec = np.vstack((maxMovementRec,maxMovement));
                                
                                if modelName != 'none': 
                                    behPredictionRec = np.vstack((behPredictionRec,predictionLabel))
                                else: behPredictionRec = 0     
                                   
                        
                       
                        dictPosRec = {"xPosRec" : xPosRec, "yPosRec" : yPosRec};

                        dictST = {"recIm3C" : recIm3C, "dictPosRec" : dictPosRec, "maxMovementRec" : maxMovementRec, "behPredictionRec" : behPredictionRec};
                        
                        nameSMRec = 'dict3C_' + fileName[0:-4] + '_fb' + str(fb) + '_bf_' + str('%06.0f' % bf) 
                       
                        if OSplatform == 'winds':           
                            #newPath = dirPathOutput + '\\' + fileName[0:-4] + '_fb' + str(fb)  #!!!!!!works with non-numbered clips (one clip per video recording)
                            newPath = dirPathOutput + '\\' + fileName[0:-6] + '_fb' + str(fb) #!!!!!!works with numbered clips (multiple clips of the same video; clip names end with index number)

                        if OSplatform == 'mac':           
                            #newPath = dirPathOutput + '/' + fileName[0:-4] + '_fb' + str(fb)  #!!!!!!works with non-numbered clips (one clip per video recording)
                            newPath = dirPathOutput + '/' + fileName[0:-6] + '_fb' + str(fb) #!!!!!!works with numbered clips (multiple clips of the same video; clip names end with index number)
 

                        if not os.path.exists(newPath):
                            os.mkdir(newPath);
                        if OSplatform == 'winds': 
                            fileDirPathOutputName = newPath + '\\' + nameSMRec;
                        if OSplatform == 'mac': 
                            fileDirPathOutputName = newPath + '/' + nameSMRec;    
                                   
                        with open(fileDirPathOutputName, "wb") as f:
                            pickle.dump(dictST, f)

                             
                    frRecSh = frRec.shape;
                    frRecRemain = frRec[bufferSize:frRecSh[0],:]
            clIndex = clIndex +1;
            print(clIndex)
            
    #return recIm3C            
                
OSplatform = 'winds';
#OSplatform = 'mac';

frameRate = 30;

clipStart = 0
clipEnd = 999 # janelia
#clipEnd = 240*frameRate;
#clipEnd = 1200*frameRate; #crimson movies #length of clip in seconds
#clipEnd = 298*frameRate; #manual_scoring #length of clip in seconds


clipsNumberMax = 51; #

fbList = [1,2,3,4]; # works for raw movies with 2x2 arenas (split the frames into 4)
#fbList = [1]; # one arena in the frame

clipFirst = 0;

bufferSize = 50;

#modelName = 'none'
#modelName = 'modelConv2ABRS_3C_train_with_labelCS1fb1_SS_plus_LiManualLabel_3'
modelName = 'modelConv2ABRS_3C_train_with_labelCS1fb1_SS_plus_LiManualLabel'

firstFolder = 0; # 

if OSplatform == 'winds':
    # path to raw video data folder; video clips must be in subfolders at this path
    #rawVidDirPath = 'C:\\Users\\ravbar\\Desktop\\ABRS_Python_GHws1\\Data_demo\\RawVideo'
    #rawVidDirPath = 'Z:\\primoz\\AVI_grooming_movies'
    #rawVidDirPath = 'Z:\\Li\\AVI_manual scoring\\videos'
    rawVidDirPath = 'C:\\Users\\primo\\Desktop\\USB\\ABRS\\ABRS_Python_GHws1\\Data_demo\\RawVideo\\';windowST = 16;

    # the output folder for ST images; subfolders will be created for each movie
    #dirPathOutput = 'C:\\Users\\ravbar\\Desktop\\ABRS_Python_GHws1\\Data\\ST';windowST = 16;
    #dirPathOutput = 'Z:\\primoz\\ABRS_python_data\\im3C_CS_janelia';windowST = 16;
    #dirPathOutput = 'Z:\\primoz\\ABRS_python_data\\im3C_crimsonLi';windowST = 16;
    dirPathOutput = 'C:\\Users\\primo\\Desktop\\USB\\ABRS\\ABRS_Python_GHws1\\Data\\im3C\\im3Ctestig';

if OSplatform == 'mac':
    # path to raw video data folder; video clips must be in subfolders at this path
    rawVidDirPath = '/Users/ravbar/Desktop/ABRS_Python_GHws1/Data_demo/RawVideo'

    # the output folder for ST images; subfolders will be created for each movie
    dirPathOutput = '/Users/ravbar/Desktop/ABRS_Python_GHws1/Data/ST';windowST = 16;    

videoFolderList = sorted(os.listdir(rawVidDirPath));
#videoFolderList = os.listdir(rawVidDirPath);
sz = np.shape(videoFolderList);sizeVideoFolder = sz[0];

for fld in range(firstFolder, sizeVideoFolder):
    
    print(fld)

    currentVideoFolder = videoFolderList[fld];

    if currentVideoFolder[-5:] != 'Store' :

        dirPathInput = rawVidDirPath + '\\' + currentVideoFolder;
        clipList = sorted(os.listdir(dirPathInput));szClipList = np.shape(clipList);
        clipsNumber = szClipList[0];

        if clipsNumber > clipsNumberMax:
            clipsNumber = clipsNumberMax;

        video_clips_to_3C_image_fun (dirPathInput,dirPathOutput,fbList,clipStart,clipEnd,clipsNumber,bufferSize,windowST,modelName,OSplatform);


0
C:\Users\primo\Desktop\USB\ABRS\ABRS_Python_GHws1\Data_demo\RawVideo\\emptychrimson_40per_2\emptychrimson_dust_40%_2.avi
C:\Users\primo\Desktop\USB\ABRS\ABRS_Python_GHws1\Data_demo\RawVideo\\emptychrimson_40per_2\emptychrimson_dust_40%_2.avi
C:\Users\primo\Desktop\USB\ABRS\ABRS_Python_GHws1\Data_demo\RawVideo\\emptychrimson_40per_2\emptychrimson_dust_40%_2.avi
C:\Users\primo\Desktop\USB\ABRS\ABRS_Python_GHws1\Data_demo\RawVideo\\emptychrimson_40per_2\emptychrimson_dust_40%_2.avi
C:\Users\primo\Desktop\USB\ABRS\ABRS_Python_GHws1\Data_demo\RawVideo\\emptychrimson_40per_2\emptychrimson_dust_40%_2.avi
C:\Users\primo\Desktop\USB\ABRS\ABRS_Python_GHws1\Data_demo\RawVideo\\emptychrimson_40per_2\emptychrimson_dust_40%_2.avi
C:\Users\primo\Desktop\USB\ABRS\ABRS_Python_GHws1\Data_demo\RawVideo\\emptychrimson_40per_2\emptychrimson_dust_40%_2.avi
C:\Users\primo\Desktop\USB\ABRS\ABRS_Python_GHws1\Data_demo\RawVideo\\emptychrimson_40per_2\emptychrimson_dust_40%_2.avi
C:\Users\primo\Desktop\USB\ABR

KeyboardInterrupt: 

In [ ]:
videoFolderList

In [ ]:
with open('C:\\Users\\primo\\Desktop\\USB\ABRS\ABRS_Python_GHws1\Data\im3C\im3Ctestig\emptychrimson_dust_40%_fb1' + '\\dict3C_emptychrimson_dust_40%_2_fb1_bf_000000', "rb") as f:
    dict3C_test = pickle.load(f)

In [ ]:
dict3C_test


In [ ]:
dirPathInput

In [ ]:
fileDirPathInputName

In [ ]:
recIm3C